In [1]:
import datetime as dt
import os, glob
import time
import numpy as np
import pandas as pd
import pathlib as pb
import cv2
from PIL import Image

### Getting Current Working Directory - CWD

In [2]:
directory = os.getcwd()
print(directory)

D:\NITESH PERSONAL\PROJECTS\FINAL PROJECT


In [3]:
def onlyalphanum(st):
    st = ''.join(e for e in st if e.isalnum())
    return st[:14]

def get_todays_folder():
    today = onlyalphanum(str(dt.date.today()))
    final_path = os.path.join(directory+"/Unidentified/"+today)
    if not os.path.exists(final_path):
        os.makedirs(final_path)
        open(final_path+'/log_file.csv', 'a');
    valid_path = pb.PureWindowsPath(final_path).as_posix()
    return valid_path

get_todays_folder()

'D:/NITESH PERSONAL/PROJECTS/FINAL PROJECT/Unidentified/20210412'

In [4]:
log_file = open(os.path.join(directory+"/Unidentified/"+onlyalphanum(str(dt.date.today())))+'/log_file.csv', 'a+');

### Importing the DB of Authorised users

In [5]:
db = pd.read_csv('database.csv', index_col='id')
db.head()

,name
id,
0,Nitesh
1,Akshat


In [6]:
class ImageIdentificationAlgorithm():
    def __init__(self):
        pass
    
    #Dataset Generator
    def generateDataset(self,name):
        global db
        if name in db['name'].values:
            print(name,"..already exists")
        else:
            id = self.updateDB(name)
            vid_cam = cv2.VideoCapture(0)
            vid_cam.set(3, 480)
            vid_cam.set(4, 360)
            face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
            count = 0;
            while True:
                check, Image_frame = vid_cam.read()
                gray = cv2.cvtColor(Image_frame, cv2.COLOR_BGR2GRAY)
                faces = face_detector.detectMultiScale(Image_frame, 1.2, 5)

                for x, y, w, h in faces:
                    cv2.rectangle(Image_frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    count += 1
                    saving_path = os.path.join(directory+"\data","user."+str(id) + "." + str(count) + ".jpg")
                    cv2.imwrite(saving_path, Image_frame)
                    cv2.imshow(str(id), Image_frame)

                if (cv2.waitKey(100)) == ord('q'):
                    break
                elif (count >= 50):
                    break

            vid_cam.release()
            cv2.destroyAllWindows()
            print(name, '... is authorised with ID -', id)
            
    #Trainer
    def train(self):
        recognizer = cv2.face_LBPHFaceRecognizer.create()
        detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

        def getImagesAndLabels(path=None):
            if path is None: path = directory+"/data"
            imagepaths = [os.path.join(path, f) for f in os.listdir(path)]
            face_samples = []
            ids = []
            for image_path in imagepaths:
                pil_img = Image.open(image_path).convert('L')
                img_numpy = np.array(pil_img, 'uint8')
                id = int(os.path.split(image_path)[-1].split(".")[1])
                faces = detector.detectMultiScale(img_numpy)
                
                for x, y, w, h in faces:
                    face_samples.append(img_numpy[y:y + h, x:x + w])
                    ids.append(id)
                    
#             print(len(ids))
            return face_samples, ids

        faces, ids = getImagesAndLabels()
        if(len(faces)==0 or len(ids)==0):
            os.remove('Trainer/trainer.yml')
            print('No training data')
            return
        recognizer.train(faces, np.array(ids))
        recognizer.save('trainer/trainer.yml')
        recognizer.write('trainer/trainer.yml')
        print("Trained Succesfully....")
        exit(1)
        
    #Recognizer
    def recognize(self):
        recognizer = cv2.face_LBPHFaceRecognizer.create()
        recognizer.read('trainer/trainer.yml')
        facecascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

        font = cv2.FONT_HERSHEY_COMPLEX_SMALL

        cam = cv2.VideoCapture(0)
        cam.set(3, 480)
        cam.set(4, 360)

        minW = int(0.1 * cam.get(3))
        minH = int(0.1 * cam.get(4))

        while True:
            check, Image_frame = cam.read()
            gray = cv2.cvtColor(Image_frame, cv2.COLOR_BGR2GRAY)

            faces = facecascade.detectMultiScale(Image_frame, 1.1, 5, minSize=(35,35))
            for x, y, w, h in faces:
                cv2.rectangle(Image_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                id, confidence = recognizer.predict(gray[y:y + h, x:x + w])
                confidence = round(100 - confidence)

                if (confidence <= 25):
                    name = "unknown_user"
                    path = str(get_todays_folder())
                    file_name = path + "/unkn_at_" + onlyalphanum(str(dt.datetime.now())) + ".jpg"                    
                    cv2.imwrite(file_name, Image_frame)

                elif (confidence < 100 and confidence > 25):
                    global db
                    name = db.loc[id][0]
#                     print(id,db.loc[id]);
                    log_file.write(name+","+str(dt.datetime.now())+"\n")

                confidence = " {0}%".format(round(100 - confidence))

                cv2.putText(Image_frame, str(name), (x + 5, y - 5), font, 1, (255, 255, 255), 2)
                cv2.putText(Image_frame, confidence, (x + 5, y + h - 5), font, 1, (255, 255, 255), 1)
                time.sleep(0.5)

            cv2.imshow('Recognizing', Image_frame)
            if cv2.waitKey(10) == ord('q'):
                break

        cam.release()
        cv2.destroyAllWindows()
        
    #Remove/Unauthorize User
    def removeUser(self, name):
        name = name.lower()
        global db
        if name in db['name'].values:
            for file in glob.iglob('Data/user.'+str(db[db['name']==name].index[0])+'*.jpg'):
                os.remove(file)        
            db = db[db['name']!=name]
            db.to_csv('database.csv')
            db = pd.read_csv('database.csv', index_col='id')                
            self.train()
        else:
            print('User doesn\'t exisit...')
    
    #Update Database
    def updateDB(self, name):
        global db
        newId = 0 if (len(db.index) == 0) else (db.loc[len(db.index)-1].name+1)
        db.loc[newId] = name.lower()
        db.to_csv('database.csv')
        db = pd.read_csv('database.csv', index_col='id')
        return newId
    
    
algo = ImageIdentificationAlgorithm()  

## Driver Code

In [32]:
algo.generateDataset('Nitesh')

Nitesh ... is authorised with ID - 0


In [33]:
algo.train()

Trained Succesfully....


In [7]:
algo.recognize()

0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object


In [8]:
algo.removeUser('nitesh')

No training data


In [9]:
while True:
    print('1. Recognize')
    print('2. Add new User')
    print('3. remove User')
    print('4. show Database')
    print('5. Exit')
    s = (int)(input())
    if(s==1):
        algo.recognize()
    elif(s==2):
        name = input("enter the name of the user")
        algo.generateDataset(name)
        algo.train()
    elif(s==3):
        name = input('enter the name of user to be removed')
        algo.removeUser(name)
    elif(s==4):
        global db
        print(db)
    elif(s==5):
        print('Exitting...')
        break

1. Recognize
2. Add new User
3. remove User
4. show Database
5. Exit


 2
enter the name of the user nitesh


nitesh ... is authorised with ID - 0
Trained Succesfully....
1. Recognize
2. Add new User
3. remove User
4. show Database
5. Exit


 1


0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
0 name    nitesh
Name: 0, dtype: object
1. Recognize
2. Add new User
3. remove User
4. show Database
5. Exit


...

      name
id        
0   nitesh
1. Recognize
2. Add new User
3. remove User
4. show Database
5. Exit


 3
enter the name of user to be removed nitesh


No training data
1. Recognize
2. Add new User
3. remove User
4. show Database
5. Exit


 5


Exitting...


In [7]:
db.head()

,name
id,
0,nitesh


In [9]:
db.loc[0][0]

'nitesh'